In [1]:
execfile('../spark-scripts/split.py')
execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')

In [ ]:
import json
import copy

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'giroCompletoTestMultipleConfs_excludeTrue'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 5
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = 'split'
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'SAGH'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']['numGH'] = 10
conf['algo']['props']['skipTh'] = 0
# ****** END EXAMPLE ****************#

# SAGH

In [ ]:
basePath = path.join('s3n://', conf['general']['bucketName'])
trackPath = path.join(basePath, '30Mdataset/entities/tracks.idomaar.gz')
trackRDD = sc.textFile(trackPath).cache()
train,test = loadDataset(conf) #loads training and test users events, then generates the train and test splits

In [ ]:
import json
def readJson(x,field='artists',pos=4):
    ##if field == "": return json.loads(x[pos])
    return json.loads(x[pos])[field]

## artistLookupRDD = (trackId, artistID)
tabSplit = lambda x: x.split("\t")
ext = lambda x: (int(x[1]), int(readJson(x)[0]['id']))
artistLookupRDD = trackRDD.map(tabSplit).map(ext).distinct().persist()
artistLookupRDD.first()

In [ ]:
## (trackId,sessionId)
ext = lambda x: ([(k['id'],k['playratio'], x['id']) for k in x['linkedinfo']['objects']])
th = conf['algo']['props']['skipTh']
batchTrainingRDD = train.flatMap(lambda x: ext(json.loads(x))).filter(lambda x: x[1] > th)\
                .map(lambda x: (int(x[0]), int(x[2]))).persist()


In [ ]:
batchTrainingRDD.take(10)

In [ ]:
## sessionId, (trackId, artistId)
joinedRDD = artistLookupRDD.join(batchTrainingRDD).map(lambda x: (x[1][1],(x[0],x[1][0]))).persist()


In [ ]:
joinedRDD.take(10)

## Greatist Hits of each artist (i.e., the top n most popular tracks for each artist)

In [ ]:
## top popular track per artist
## trackId_artistId
from operator import add
parser = lambda x: (x[1], 1)
prep = lambda x: (x[0][1], [(x[0][0], x[1])])

def sorter(x,n):
    if type(x[1]) != list: 
        result = list()
        result.append(x[1])
        return result
    a = x[1]
    if len(a) <= n: n=len(a)
    return (x[0],sorted(a,key=lambda k: -k[1])[0:n])

numGH = conf['algo']['props']['numGH']
sort = lambda x: sorter(x, numGH)
uni = lambda x: x[1] > 1

artistGreatistHitsRDD = joinedRDD.map(parser).reduceByKey(add).filter(uni).map(prep).reduceByKey(add).map(sort)

In [ ]:
artistGreatistHitsRDD.take(10)

## Recommendation:  parsing the request

In [ ]:
ext2 = lambda x: [(int(x['id']), int(l['id']), l['playratio']) for l in x['linkedinfo']['objects']]
recReqRDD =  test.flatMap(lambda x: ext2(json.loads(x)))\
            .filter(lambda x: x[2] > th).map(lambda x: (x[1], x[0])).join(artistLookupRDD)\
            .map(lambda x: ((x[1][1], x[1][0]), [(x[0])])).reduceByKey(add)\
            .map(lambda x: (x[0][0], (x[0][1], x[1])))


In [ ]:
recReqRDD.take(10)

## SAGH

In [ ]:
joinedRec = recReqRDD.join(artistGreatistHitsRDD).persist()
joinedRec.count()

In [ ]:
def validator(x):
    recID = x[1][0][0]
    trackList = x[1][0][1]
    recList = x[1][1]
    rec = [l for l in recList if l[0] not in trackList]
    return (recID,rec)

val = lambda x: validator(x)

def sorter(x,length):
    a = x[1]
    if len(a) > length: length = len(a) 
    return sorted(a,key=lambda k: -k[1])[0:length]

recLength = conf['split']['reclistSize']
s = lambda x: (x[0], sorter(x, recLength))

rec = joinedRec.map(val).reduceByKey(add).map(s)

In [ ]:
rec.take(10)

In [ ]:
#Generate recommendations in the right format, then test
def addResponse(reqJson, respJson):
    req = json.loads(reqJson)
    req['linkedinfo'].update({'response': respJson})
    return json.dumps(req)

def formatResponse(rank_list, response_size=100):
    return [{'type': 'track', 'id': id, 'rank': rank} for rank, (id, _) in enumerate(rank_list[:response_size])]

responses = test.map(lambda x: (json.loads(x)['id'], x))\
    .join(rec)\
    .map(lambda x: addResponse(x[1][0], formatResponse(x[1][1], json.loads(x[1][0])['properties']['reclistSize'])))

In [ ]:
saveRecommendations(conf, responses, overwrite=True)

In [ ]:
computeMetrics(conf)